In [1]:
import MDAnalysis as md
import numpy as np
import matplotlib.pyplot as plt
import string
import os
import argparse


In [15]:
def extract_no_beads_bulk (u):
    """Function to count the number of beads of the different lipid types for a random binding site in the bulk.
    The volume around the BS is approximated by a cylinder with radius = 7 Å and height = 14 Å"""
    prot = u.select_atoms('name BB SC1 SC2 SC3 SC4')
    prot_rdf = prot.radius_of_gyration()+20
    prot_com = prot.center_of_mass()

    # Select the protein zone
    prot = u.select_atoms('name BB SC1 SC2 SC3 SC4')
    protein_zone = u.select_atoms('cyzone {0:f} {1:f} {2:f} group protein'
                  .format(prot_rdf, 60, -60), protein=prot, updating=True)

    #Select the bulk zone - not overlapping with the protein zone
    bulk_selection = u.select_atoms('all and not protein and not group protein_zone'
                                    .format(prot_rdf, 60, -60), protein=prot, protein_zone=protein_zone, updating=True)

    # Select a random bead in the bulk selection, as a virtual binding site
    idx_random = np.random.randint(0,bulk_selection.positions.shape[0])
    ran_bs = bulk_selection.indices[idx_random]
    x_ran, y_ran, z_ran = bulk_selection.positions[idx_random]

    # Extract the number of beads around the site
    atom_sel = bulk_selection.select_atoms('prop x == {0:f} and prop y == {1:f} and prop z == {2:f}'.format(x_ran, y_ran, z_ran))
    try:
        no_beads_bs_all = u.select_atoms('(cyzone {0:f} {1:f} {2:f} group atom_sel) and not group protein'.format(7, 7, -7), atom_sel=atom_sel, protein=prot)
    except ValueError:
        print (no_beads_bs_all)
        print ("(cyzone {0:f} {1:f} {2:f} group atom_sel) and not group protein".format(7, 7, -7))

    res = no_beads_bs_all.resnames
    unique, counts = np.unique(res, return_counts=True)

    #Returns the unique lipids found around the BS along with the number of beads for each
    return unique, counts


In [3]:
def assign_counts (u, unique, counts):
    """Function for assigning the counts (of beads) for each
    lipid type to an array."""
    memb = u.select_atoms('all and not (name BB SC1 SC2 SC3 SC4)').resnames
    unique_lipid, counts_lipid = np.unique(memb, return_counts=True)
    zeros = np.zeros([unique_lipid.shape[0]])
    counts_out = np.vstack((unique_lipid, zeros))

    if len(counts) != 0:
        idxs = [ np.where(counts_out[0,:] == i)[0] for i in unique ]
        shape = len(idxs)
        try:
            counts_out[1,idxs] = counts.reshape([shape,1])
        except IndexError:
            print ('############')
            print (counts)
            print (unique)
            print (idxs)
            print('#############')
            raise
    return counts_out


In [4]:
def main_func (u, stride=10):
    """Main function
    Iterative over the systems, calling only this function."""
    ### MAIN ###

    #Set selections
    prot = u.select_atoms('name BB SC1 SC2 SC3 SC4')
    print ("protein selected")
    prot_rdf = prot.radius_of_gyration()+20
    prot_com = prot.center_of_mass()
    membrane = u.select_atoms('all and not (name BB SC1 SC2 SC3 SC4)')
    print ("membrane selected")
    protein_zone = u.select_atoms('cyzone {0:f} {1:f} {2:f} group protein'.format(prot_rdf, 60, -60), protein=prot, updating=True)
    print ("protein zone selected")
    memb_prot_zone = protein_zone.select_atoms('all and not group protein', protein=prot, updating=True)
    print ("membrane zone selected")
    bulk_selection = u.select_atoms('all and not group protein and not group protein_zone'
                                    .format(prot_rdf, 60, -60), protein=prot, protein_zone=protein_zone, updating=True)
    print ("Bulk selected")
    TMD = u.select_atoms('group protein and resid 111-394', protein=prot) #### UPDATE THIS
    print ("TMD selected as group protein and resid 111-394")
    TMD_resid = np.unique(TMD.resids)
    print ('Selections set')

    output_counts_all = []
    output_counts_all_bulk = []

    for idx, ts in enumerate(u.trajectory[::stride]):
        counts_TMD_local = []
        unique_bulk, counts_bulk = extract_no_beads_bulk (u)
        print ("Counts extracted for bulk")
        out_bulk_local = assign_counts(u, unique_bulk, counts_bulk)
        output_counts_all_bulk.append(out_bulk_local)
        for indx, r in enumerate(TMD_resid):
            sel = u.select_atoms('(cyzone {0:f} {1:f} {2:f} resid {3:d}) and not group protein'
                                                .format(7, 7,-7, r), protein=prot)
            sel_res = sel.resnames
            unique, counts = np.unique(sel_res, return_counts=True)
            out_local = assign_counts(u, unique, counts)

            counts_TMD_local.append(out_local)
        output_counts_all.append(np.array(counts_TMD_local))

    COUNTS_PROT = np.array(output_counts_all) # [nframes, nresids, 2, nlipid_types]
    COUNTS_VIRTUAL_BS = np.array(output_counts_all_bulk) # [nframes, 2, nlipid_types]
    return COUNTS_PROT, COUNTS_VIRTUAL_BS


In [5]:
system=['IA']

In [ ]:
for idx, s in enumerate(system):
    for r in range(1):
        gro = 'GRO/{0:s}_{1:d}_nowat.gro'.format(s,r)
        xtc = 'XTC/{0:s}_{1:d}_nowat_res_center_fit_last5.xtc'.format(s,r)
        print ("Loaded Universe with: GRO/{0:s}_{1:d}_nowat.gro XTC/{0:s}_{1:d}_nowat_res_center_last5.xtc".format(s,r))
        u = md.Universe(gro,xtc)
        print ("Universe loaded with GRO/{0:s}_{1:d}_nowat.gro XTC/{0:s}_{1:d}_nowat_res_center_last5.xtc".format(s,r))
        if os.path.isdir("CONTACT_NO_BEADS") == False:
                os.system("mkdir CONTACT_NO_BEADS")
                print ("Output will be saved in the directory called CONTACT_NO_BEADS")
        else:
                print ("Output will be saved in the directory called CONTACT_NO_BEADS")
        output_dat1 = 'CONTACT_NO_BEADS/{0:s}_{1:d}_number_of_beads_protein.npy'.format(s,r)
        output_dat2 = 'CONTACT_NO_BEADS/{0:s}_{1:d}_number_of_beads_bulk.npy'.format(s,r)
        if os.path.exists(output_dat1):
            print ("{0:s} exists".format(output_dat1))
            continue
        else:
            print ("Performing calculation")
            COUNTS_PROT, COUNTS_VIRTUAL_BS = main_func (u)
            np.save(output_dat1, COUNTS_PROT)
            np.save(output_dat2, COUNTS_VIRTUAL_BS)



#### TESTING

In [6]:
s = 'IA'
r = 0

In [7]:
gro = 'GRO/{0:s}_{1:d}_nowat.gro'.format(s,r)
xtc = 'XTC/{0:s}_{1:d}_nowat_res_center_fit_last5.xtc'.format(s,r)
print ("Loaded Universe with: GRO/{0:s}_{1:d}_nowat.gro XTC/{0:s}_{1:d}_nowat_res_center_last5.xtc".format(s,r))
u = md.Universe(gro,xtc)

Loaded Universe with: GRO/IA_0_nowat.gro XTC/IA_0_nowat_res_center_last5.xtc


/home/au447022/anaconda3/lib/python3.7/site-packages/MDAnalysis/topology/guessers.py:80: UserWarning: Failed to guess the mass for the following atom types: A
  warnings.warn("Failed to guess the mass for the following atom types: {}".format(atom_type))
/home/au447022/anaconda3/lib/python3.7/site-packages/MDAnalysis/topology/guessers.py:80: UserWarning: Failed to guess the mass for the following atom types: D
  warnings.warn("Failed to guess the mass for the following atom types: {}".format(atom_type))
/home/au447022/anaconda3/lib/python3.7/site-packages/MDAnalysis/topology/guessers.py:80: UserWarning: Failed to guess the mass for the following atom types: G
  warnings.warn("Failed to guess the mass for the following atom types: {}".format(atom_type))
/home/au447022/anaconda3/lib/python3.7/site-packages/MDAnalysis/topology/guessers.py:80: UserWarning: Failed to guess the mass for the following atom types: R
  warnings.warn("Failed to guess the mass for the following atom types: {}".for

In [8]:
prot = u.select_atoms('name BB SC1 SC2 SC3 SC4')
print ("protein selected")
prot_rdf = prot.radius_of_gyration()+20
prot_com = prot.center_of_mass()
membrane = u.select_atoms('all and not (name BB SC1 SC2 SC3 SC4)')
print ("membrane selected")
protein_zone = u.select_atoms('cyzone {0:f} {1:f} {2:f} group protein'.format(prot_rdf, 60, -60), protein=prot, updating=True)
print ("protein zone selected")
memb_prot_zone = protein_zone.select_atoms('all and not group protein', protein=prot, updating=True)
print ("membrane zone selected")
bulk_selection = u.select_atoms('all and not group protein and not group protein_zone'
                                .format(prot_rdf, 60, -60), protein=prot, protein_zone=protein_zone, updating=True)
print ("Bulk selected")
TMD = u.select_atoms('group protein and resid 111-394', protein=prot) #### UPDATE THIS
print ("TMD selected as group protein and resid 111-394")
TMD_resid = np.unique(TMD.resids)
print ('Selections set')

output_counts_all = []
output_counts_all_bulk = []



protein selected
membrane selected
protein zone selected
membrane zone selected
Bulk selected
TMD selected as group protein and resid 111-394
Selections set


In [16]:
stride = 1000

for idx, ts in enumerate(u.trajectory[::stride]):
    #print ('time',ts.time)
    print ('frame', ts.frame)
    counts_TMD_local = []
    unique_bulk, counts_bulk = extract_no_beads_bulk (u)
    #print ("Counts extracted for bulk")
    out_bulk_local = assign_counts(u, unique_bulk, counts_bulk)
    output_counts_all_bulk.append(out_bulk_local)
    for indx, r in enumerate(TMD_resid):
        #print ('resid', r)
        sel = u.select_atoms('(cyzone {0:f} {1:f} {2:f} resid {3:d}) and not group protein'
                                            .format(7, 7,-7, r), protein=prot)
        #print ('selection made')
        sel_res = sel.resnames
        unique, counts = np.unique(sel_res, return_counts=True)
        out_local = assign_counts(u, unique, counts)

        counts_TMD_local.append(out_local)
    output_counts_all.append(np.array(counts_TMD_local))

COUNTS_PROT = np.array(output_counts_all) # [nframes, nresids, 2, nlipid_types]
COUNTS_VIRTUAL_BS = np.array(output_counts_all_bulk) # [nframes, 2, nlipid_types]

frame 0
frame 1000
frame 2000
frame 3000
frame 4000
frame 5000
frame 6000
frame 7000
frame 8000
frame 9000
frame 10000
frame 11000


In [17]:
np.array(output_counts_all_bulk).shape

(60, 2, 9)

In [18]:
np.array(output_counts_all).shape

(59, 284, 2, 9)